In [417]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [418]:
data1 = pd.read_excel('U-GENE_data_RU_PL_fixed (1).xlsx')
data1 = data1[0:-1].drop(columns={'Unnamed: 0'})

In [419]:
data1['Diabetes'] = data1['Diabetes'].replace(2,1)

In [420]:
data1['Increased fasting glucose'] = data1['Increased fasting glucose'].replace(2,1)

In [421]:

data1 = data1.replace('<25', 25)
data1 = data1.replace('bd', 31)
data1 = data1.replace('ХИЛЕЗ', 4.12)
data1 = data1.replace('>350', 352)
data1 = data1.replace('`1', 1)
data1 = data1[~data1['Ethnicity'].isin([1, 2, 3, 4])].reset_index(drop=True)
data1['Ethnicity'] = data1['Ethnicity'].replace(5,1)

In [422]:
potenial_targets = ['Vascular diseases ',
       'Impared fasting glucose (prediabetes)', 'Diabetes',
       'Heart attack, chest pain, or other heart condition', 'Thyroid disease',
       'Dyslipidemia', 'Increased triglycerides', 'High blood pressure',
       'Increased fasting glucose', 'Metabolic Syndrome', 'Condition']

In [423]:
data1['High blood pressure'].unique()

array([ 1.,  0., nan])

In [424]:
def change_target_dom(x):
    if x == 3:
        return 1
    elif x == 2:
        return 1
    else:
        return 0 

In [425]:
genetics = ['UCP1 rs45539933 (+1068GA)', 'UCP1 rs1800592  (-3826AG)',
       'UCP1 rs3811791 (-1766AG)', 'UCP1 rs10011540 (-112AC)',
       'UCP1 rs2270565 (Met229Leu)', 'UCP2 rs659366 (-866GA)', 'UCP2 rs660339 (Ala55Val)',
       'UCP3 rs1800849', 'FTO rs9939609']

In [426]:
for el in genetics:
    data1[el] = data1[el].apply(lambda x: change_target_dom(x))

In [427]:
data_to_learn = data1[[i for i in data1.columns if i not in potenial_targets]].copy()

In [428]:
only_genetics = data1[['UCP1 rs45539933 (+1068GA)', 'UCP1 rs1800592  (-3826AG)',
       'UCP1 rs3811791 (-1766AG)', 'UCP1 rs10011540 (-112AC)',
       'UCP1 rs2270565 (Met229Leu)', 'UCP2 rs659366 (-866GA)',
       'UCP2 rs660339 (Ala55Val)', 'UCP3 rs1800849', 'FTO rs9939609', 'Metabolic Syndrome']].copy()

In [429]:
only_genetics = only_genetics.dropna()

In [430]:
only_genetics

,UCP1 rs45539933 (+1068GA),UCP1 rs1800592 (-3826AG),UCP1 rs3811791 (-1766AG),UCP1 rs10011540 (-112AC),UCP1 rs2270565 (Met229Leu),UCP2 rs659366 (-866GA),UCP2 rs660339 (Ala55Val),UCP3 rs1800849,FTO rs9939609,Metabolic Syndrome
0,0,0,0,0,0,1,1,0,1,1.0
1,0,1,1,0,0,0,0,0,0,1.0
2,0,0,0,0,0,1,1,0,1,1.0
3,0,1,1,0,0,0,0,0,0,1.0
4,0,1,0,0,0,1,1,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...
1175,1,1,0,1,1,0,0,0,0,0.0
1176,0,1,0,0,0,1,1,1,0,0.0
1177,0,0,0,0,0,1,1,1,0,0.0
1178,0,0,0,0,0,0,0,0,1,0.0


In [431]:
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

# test p-values
x = only_genetics.drop(columns=['Metabolic Syndrome'])
y = only_genetics['Metabolic Syndrome']
model = LogisticRegression().fit(x, y)



# compare with statsmodels
import statsmodels.api as sm
sm_model = sm.Logit(y, sm.add_constant(x)).fit(disp=0)
print(sm_model.pvalues)
sm_model.summary()

const                         2.537446e-11
UCP1 rs45539933 (+1068GA)     4.482997e-01
UCP1 rs1800592  (-3826AG)     3.274845e-01
UCP1 rs3811791 (-1766AG)      4.541584e-02
UCP1 rs10011540 (-112AC)      1.397867e-01
UCP1 rs2270565 (Met229Leu)    6.756009e-01
UCP2 rs659366 (-866GA)        8.705162e-01
UCP2 rs660339 (Ala55Val)      7.733995e-01
UCP3 rs1800849                2.289741e-03
FTO rs9939609                 2.810933e-01
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     Metabolic Syndrome   No. Observations:                  992
Model:                          Logit   Df Residuals:                      982
Method:                           MLE   Df Model:                            9
Date:                Sun, 02 Jun 2024   Pseudo R-squ.:                 0.01581
Time:                        21:00:26   Log-Likelihood:                -607.15
converged:                       True   LL-Null:                       -616.90
Covariance Type:            nonrobust   LLR p-value:                   0.02122
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         -1.2358      0.185     -6.671      0.000      -1.599      -0.873
UCP1 rs45539933 (+1068GA)      0.5058      0.667      0.758      0.448      -0.802       1.813
UCP1 rs1800592  (-3826AG)      0.1488      0.152      0.979      0.327      -0.149       0.447
UCP1 rs3811791 (-1766AG)       1.7114      0.855      2.001      0.045       0.035       3.388
UCP1 rs10011540 (-112AC)      -0.7360      0.498     -1.477      0.140      -1.713       0.241
UCP1 rs2270565 (Met229Leu)     0.2065      0.494      0.418      0.676      -0.761       1.174
UCP2 rs659366 (-866GA)        -0.0534      0.328     -0.163      0.871      -0.696       0.589
UCP2 rs660339 (Ala55Val)       0.0954      0.331      0.288      0.773      -0.554       0.744
UCP3 rs1800849                 0.4281      0.140      3.050      0.002       0.153       0.703
FTO rs9939609                  0.1574      0.146      1.078      0.281      -0.129       0.444
==============================================================================================
"""